In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import json
import logging
from typing import Dict, List, Tuple
import mlflow
from datetime import datetime

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('erp_bot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [3]:
class ERPDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [4]:
class ERPIntentClassifier(nn.Module):
    def __init__(self, n_classes):
        super.__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(768, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask
        )
        pooled_output = outputs[1]
        output = self.drop(pooled_output)
        return self.fc(output)

In [ ]:
class IntentClassifier:
    def __init__(self, model_path: str = None):
        self.device = torch.device('cpu')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model_path = None
        self.label_to_id = {}
        if model_path:
            self.load_model(model_path)

    def load_model(self, path: str):
        checkpoint = torch.load(path, map_location = self.device)
        self.model = ERPIntentClassifier(len(checkpoint['label_to_id']))
        self.model.load_state_dict(checkpoint['model_state'])
        self.label_to_id = checkpoint['label_to_id']
        self.model.to(self.device)
        logger.info(f"Model loader from {path}")

    def save_model(self, path: str):
        torch.save({
            'model_state': self.model.state_dict(),
            'label_to_id': self.label_to_id
        }, path) 
        logger.info(f"Model saved to {path}")

    def train(self, train_data: Dict[str, List[str]], epochs = 5, batch_size: int = 16):
        texts, labels = [], []
        for intent, examples in train_data.items():
            if intent not in self.label_to_id:
                self.label_to_id[intent] = len(self.label_to_id)
            text.extend(examples)
            labels.extend([self.label_to_id[intent]]*len(examples))
    
        dataset = ERPDataset(texts, labels, self.tokenizer)
        dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
        self.model = ERPIntentClassifier(len(self.label_to_id))
        self.model.to(self.device)
        optimizer = torch.optim.AdamW(self.model.parameters(), lr = 2e-5)
        criterion = nn.CrossEntropyLoss()

        mlflow.start_run()
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in dataloader:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['label'].to(self.device)
                outputs = self.model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(dataloader)
            logger.info(f"Epoch: {epoch + 1} / {epochs}, Loss: {avg_loss:.4f}")
            mlflow.log_metric("loss", avg_loss, step = epoch)

        mlflow.end_run()

    def predict(self, text: str) -> Tuple [str, float]:
        self.model.eval()
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = 128,
            padding = 'max_length',
            truncation = True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        with torch.no_grad():
            outputs = seld.model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim = 1)
            confidence, predicted = torch.max(probabilities, 1)

        id_to_
        
                
            